In [1]:
from preprocessors.twitter_preprocessor import *
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
training = pd.read_csv("../data/emoticon_paper/training.csv", engine="python", delimiter=",", usecols=[5,0], encoding="iso-8859-1", names=["sentiment","full_text"], dtype={"sentiment":int, "full_text":str})
training.head()

,sentiment,full_text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [3]:
training["sentiment"]=training["sentiment"].parallel_apply(lambda x: int(1) if x==int(4) else x)

In [4]:
training["sentiment"].value_counts()

0    800000
1    800000
Name: sentiment, dtype: int64

In [5]:
training["prep_text"]=training["full_text"].parallel_apply(lambda x: str(TwitterPreprocessor().preprocess_tweet(x, lemmatize=False)))

In [7]:
training["prep_text_lemmatized"]=training["full_text"].parallel_apply(lambda x: str(TwitterPreprocessor().preprocess_tweet(x, lemmatize=True)))

In [11]:
training.to_csv("training_paper.csv", index=False, quoting=csv.QUOTE_ALL)

In [23]:
evaluation = pd.read_csv("../data/emoticon_paper/evaluation.csv", engine="python", delimiter=",", usecols=[5,0], encoding="iso-8859-1", names=["sentiment","full_text"], dtype={"sentiment":int, "full_text":str})

In [25]:
evaluation["sentiment"].value_counts()

4    182
0    177
2    139
Name: sentiment, dtype: int64

In [26]:
evaluation["sentiment"]=evaluation["sentiment"].parallel_apply(lambda x: int(1) if x==int(4) else x)
evaluation["sentiment"].value_counts()

1    182
0    177
2    139
Name: sentiment, dtype: int64

In [28]:
evaluation=evaluation[evaluation["sentiment"]!=int(2)]
print(evaluation.shape)
print(evaluation["sentiment"].value_counts())
evaluation.head()

(359, 2)
1    182
0    177
Name: sentiment, dtype: int64


,sentiment,full_text
0,1,@stellargirl I loooooooovvvvvveee my Kindle2. ...
1,1,Reading my kindle2... Love it... Lee childs i...
2,1,"Ok, first assesment of the #kindle2 ...it fuck..."
3,1,@kenburbary You'll love your Kindle2. I've had...
4,1,@mikefish Fair enough. But i have the Kindle2...


In [29]:
evaluation["prep_text"]=evaluation["full_text"].parallel_apply(lambda x: str(TwitterPreprocessor().preprocess_tweet(x, lemmatize=False)))
evaluation["prep_text_lemmatized"]=evaluation["full_text"].parallel_apply(lambda x: str(TwitterPreprocessor().preprocess_tweet(x, lemmatize=True)))

In [ ]:
evaluation.to_csv("evaluation_paper.csv", index=False, quoting=csv.QUOTE_ALL)